In [17]:
import torch
from torchvision.transforms import v2
from torchvision.io import read_image

In [18]:
print(f"torch version is {torch.__version__}")
print(f"is CUDA available : {torch.cuda.is_available()}")
print(f"number of available threads : {torch.get_num_threads()}")

torch version is 2.3.0+cu121
is CUDA available : True
number of available threads : 6


In [19]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [20]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from PIL import Image
import cv2

# Machine Images

## Step 1 : Roberts Edge Detection

In [21]:
import cv2  
import numpy as np 
from scipy import ndimage 
import os 

In [6]:
from tqdm import tqdm

In [22]:
def torch_preprocessing_machine(img):
    transform = torchvision.transforms.Resize(size=(810,1080))
    img = transform(img)
    transforms = v2.Compose([
    #v2.RandomHorizontalFlip(p=0.5),
    #v2.RandomVerticalFlip(p=0.5),
    #v2.functional.autocontrast(0.5),
    v2.ToDtype(torch.float64, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    v2.ColorJitter(brightness=1,contrast=0.5,saturation=0.5),
    #v2.CenterCrop(size=(810, 810)),
    #v2.RandomSolarize(threshold=0.9),
    ])
    print(img.shape)
    img2 = transforms(img)
    img2 = v2.functional.to_grayscale(img2)
    #sharpen = v2.functional.adjust_sharpness(img2,sharpness_factor=1.0)
    adj_con = v2.functional.adjust_contrast(inpt=img2, contrast_factor = 1)
    adj_bri = v2.functional.adjust_brightness(inpt= adj_con, brightness_factor = 1)
    adj_sat = v2.functional.adjust_saturation(inpt= adj_bri, saturation_factor = 0.5)
    plt.imshow(img2.permute(1, 2, 0), cmap="gray",aspect=1)
    return img2

In [39]:
def gen_robert_img(img_path):
    #print(img_path)
    path = img_path.split("\\")[-1]
    #print(path)
    roberts_cross_v = np.array( [[1, 0 ], 
                             [0,-1 ]] ) 
  
    roberts_cross_h = np.array( [[ 0, 1 ], 
                                [ -1, 0 ]] ) 
    
    img = cv2.imread(img_path,0).astype('float64') 
    img/=255.0
    #print(img.shape)
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    vertical = ndimage.convolve( img, roberts_cross_v ) 
    horizontal = ndimage.convolve( img, roberts_cross_h ) 
    #plt.imshow(img,cmap="gray")
    
    edged_img = np.sqrt( np.square(horizontal) + np.square(vertical)) 
    edged_img*=255
    path_save = r"dataset_peak_1\test"
    save_name = path
    path_save2 = os.path.join(path_save,save_name)
    #print(path_save2)
    #plt.imshow(edged_img,cmap='gray')
    cv2.imwrite(path_save2,edged_img)

#cv2.imwrite("output192.jpg",gen_robert_img(path_class_0))

In [41]:
path_os = r"liver-ultrasound-detection\test\test\images"
img_list = os.listdir(r"liver-ultrasound-detection\test\test\images")

In [42]:
def gen_images(img_list, path_os):
    for i in tqdm(img_list):
        path1 = os.path.join(path_os,i)
        x = gen_robert_img(path1)

In [43]:
#gen_images(img_list, path_os)

100%|██████████| 5153/5153 [32:27<00:00,  2.65it/s]


## Step 2 : Image Augmentation

In [44]:
from torchvision.utils import save_image
from torchvision.transforms import functional
from PIL import Image

In [349]:
aug2_list = os.listdir(r"dataset_peak_1\test")

In [354]:
def augmentation(img_path):
    img = read_image(img_path)
    name = img_path.split(os.sep)
    #print(name)
    adj_con = v2.functional.adjust_contrast(inpt=img, contrast_factor = 10)
    adj_bri = v2.functional.adjust_brightness(inpt= adj_con, brightness_factor = 10)
    adj_sat = v2.functional.adjust_saturation(inpt= adj_bri, saturation_factor = 10)
    x = functional.to_pil_image(adj_sat)
    save_path = os.path.join(r"Final_dataset_peak\test",name[-1])
    #print(save_path)
    x.convert('RGB')
    #x.show()
    x.save(save_path)


In [355]:
import torch
from torchvision.transforms import v2
from torchvision.io import read_image
import torchvision

In [356]:
def torch_preprocessing(path):
    img = read_image(path)
    transform = torchvision.transforms.Resize(size=(810,1080))
    img = transform(img)
    img3 = img/255
    mean = img3.reshape(3,-1).mean(axis=1)
    std  =  img3.reshape(3,-1).std(axis=1)
    transforms = v2.Compose([
    #v2.RandomHorizontalFlip(p=0.5),
    #v2.RandomVerticalFlip(p=0.5),
    #v2.functional.autocontrast(0.5),
    v2.ToDtype(torch.float64, scale=True),
    v2.Normalize(mean=mean, std=std),
    #v2.ColorJitter(brightness=1,contrast=0.5,saturation=0.5),
    #v2.CenterCrop(size=(810, 810)),
    #v2.RandomSolarize(threshold=0.9),
    ])
    print(img.shape)
    img2 = transforms(img)
    img2 = v2.functional.to_grayscale(img2)
    #sharpen = v2.functional.adjust_sharpness(img2,sharpness_factor=1.0)
    img3 = torchvision.transforms.functional.invert(img2)
    adj_con = v2.functional.adjust_contrast(inpt=img3, contrast_factor = 0.65)
    adj_bri = v2.functional.adjust_brightness(inpt= img2, brightness_factor = 0.8)
    adj_sat = v2.functional.adjust_saturation(inpt= adj_bri, saturation_factor = 0.5)
    plt.imshow(adj_bri.permute(1, 2, 0), cmap="gray",aspect=1)
    return img

In [357]:
augmentation(r"dataset_peak_1\test\195.jpg")

In [358]:
augmentation(r"dataset_peak_1\test\1.jpg")

In [359]:
img_path = r"dataset_peak_1\test"

In [360]:
def gen_images2(img_path, img_list):
    for i in tqdm(img_list):
        path = os.path.join(img_path, i)
        augmentation(path)



In [361]:
gen_images2(img_path,aug2_list)

100%|██████████| 5153/5153 [09:57<00:00,  8.62it/s]


# Mobile Images

## Step 1: Roberts Edge Detection

In [ ]:
def torch_preprocessing_mobile(img):
    transform = torchvision.transforms.Resize(size=(810,1080))
    img = transform(img)
    transforms = v2.Compose([
    #v2.RandomHorizontalFlip(p=0.5),
    #v2.RandomVerticalFlip(p=0.5),
    #v2.functional.autocontrast(0.5),
    v2.ToDtype(torch.float64, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    v2.ColorJitter(brightness=1,contrast=0.5,saturation=0.5),
    #v2.CenterCrop(size=(810, 810)),
    #v2.RandomSolarize(threshold=0.9),
    ])
    print(img.shape)
    img2 = transforms(img)
    img2 = v2.functional.to_grayscale(img2)
    #sharpen = v2.functional.adjust_sharpness(img2,sharpness_factor=1.0)
    adj_con = v2.functional.adjust_contrast(inpt=img2, contrast_factor = 1)
    adj_bri = v2.functional.adjust_brightness(inpt= adj_con, brightness_factor = 1)
    adj_sat = v2.functional.adjust_saturation(inpt= adj_bri, saturation_factor = 0.5)
    plt.imshow(img2.permute(1, 2, 0), cmap="gray",aspect=1)
    return img2

In [ ]:
path_os = r"sample_liver-ultrasound\train\images"
img_list = os.listdir(r"sample_liver-ultrasound\train\images")

In [ ]:
gen_images(img_list, path_os)

## Step 2: Image Augmentation

In [ ]:
aug2_list = os.listdir(r"augmented_dataset_peak\train")

In [ ]:
def augmentation_mobile(img_path):
    img = read_image(img_path)
    name = img_path.split(os.sep)
    adj_con = v2.functional.adjust_contrast(inpt=img, contrast_factor = 100)
    adj_bri = v2.functional.adjust_brightness(inpt= adj_con, brightness_factor = 20)
    adj_sat = v2.functional.adjust_saturation(inpt= adj_bri, saturation_factor = 100)
    x = functional.to_pil_image(adj_sat)
    save_path = os.path.join(r"final_aug\train",name[2])
    #print(save_path)
    x.convert('RGB')
    x.save(save_path)


In [ ]:
def gen_images2(img_path, img_list):
    for i in tqdm(img_list):
        path = os.path.join(img_path, i)
        augmentation(path)

In [ ]:
gen_images2(img_path,aug2_list)